In [14]:
from pathlib import Path
import cv2

input_dir = Path('/storage_bizon/bizon_imagedata/naravich/Unlabeled_OCT/')
output_dir = Path('/storage_bizon/naravich/Unlabeled_OCT_videos/')
output_dir.mkdir(exist_ok=True)

for patient_dir in input_dir.iterdir():
    sample_image = None
    if patient_dir.is_dir():
        images = list(patient_dir.glob('*.png'))
        if len(images) == 0:
            print(f'No images in {patient_dir}')
            break
        images.sort(key = lambda x: int(x.stem))
        
        video_name = output_dir / f'{patient_dir.stem}.mp4'
        fps = 1
        if sample_image is None:
            sample_image = list(patient_dir.glob('*.png'))[0]
            sample_image = cv2.imread(str(sample_image), cv2.IMREAD_GRAYSCALE)

        video = cv2.VideoWriter(str(video_name), cv2.VideoWriter_fourcc(*'mp4v'), fps, sample_image.shape, isColor=True)
        min_val, max_val = float('inf'), float('-inf')
        for image_path in images:
            image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            min_val = min(min_val, image.min())
            max_val = max(max_val, image.max())
            if image.shape[:2] != sample_image.shape:
                raise ValueError(f'Image {image_path} has shape {image.shape}, expected {sample_image.shape}')
            video.write(image)
        video.release()

        print(f'Processed {patient_dir.stem} ({len(images)} images), min: {min_val}, max: {max_val}')

Processed 03-003-Pre-MEASREV (539 images), min: 0, max: 127
Processed 701-005-LAD-Final-MEASREV (374 images), min: 0, max: 127
Processed 702-005-LAD-Final-MEASREV (375 images), min: 0, max: 127
Processed 101-014-RCA-Post-Lith-MEASREV (375 images), min: 0, max: 127
Processed 705-001-RCA-Final-MEASREV (375 images), min: 0, max: 127
Processed 707-004-RCA-Post-IVL-MEASREV (375 images), min: 0, max: 127
Processed 101-051-RCA-Pre-MEASREV (375 images), min: 0, max: 127
Processed 706-009-RCA-Final-MEASREV (375 images), min: 0, max: 127
Processed 135-003-LAD-Pre-MEASREV (374 images), min: 0, max: 127
Processed 02-005-Pre-MEASREV (269 images), min: 0, max: 127
Processed 101-054-LAD-Final-Dis-MEASREV (375 images), min: 0, max: 127
Processed 708-003-LAD-Final-MEASREV (375 images), min: 0, max: 127
Processed 403-002-LAD-Post-Lith-MEASREV (375 images), min: 0, max: 127
Processed 03-001-Post-Stent-MEASREV (539 images), min: 0, max: 127
Processed 101-053-LAD-Post-Lith-MEASREV (375 images), min: 0, max

In [18]:
with open(output_dir / 'Unlabel_OCT_Video.csv', 'w') as f:
    videos = list(output_dir.glob('*.mp4'))
    content = ', 0\n'.join([str(video) for video in videos])
    content += ', 0' # Add an empty line at the end
    f.write(content)